In [ ]:
!pip install azure-storage-blob

In [ ]:
import logging
import os
import glob
import json
import csv
import time
import zipfile

import pandas as pd
import numpy as np

from  azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from google.colab import userdata
from google.colab import files


In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)


In [ ]:
def download_test_data(blob_names=[]):
  # Replace with your Azure Storage connection string
  connection_string = userdata.get("YOUR_AZURE_STORAGE_CONNECTION_STRING")

  # Replace with the name of your container and the blob name you want to download
  container_name = userdata.get("YOUR_CONTAINER_NAME")
  blob_names = ["appspecificdata.csv"]

  # Create a BlobServiceClient object from the connection string
  blob_service_client = BlobServiceClient.from_connection_string(connection_string)

  for blob_name in blob_names:
    # Get a reference to the target blob
    blob_client = blob_service_client.get_blob_client(container_name, blob_name)
    # Download the blob content to a file
    with open(blob_name, 'wb') as downloaded_file:
      downloaded_file.write(blob_client.download_blob().readall())
    print('File downloaded successfully!')


# Assuming you have a DataFrame named 'df'
def create_text_file(row, typeofrow):
    row_data = ''.join(row.astype(str))
    file_path = f"/content/{typeofrow}/file_{row.name}.txt"

    # Create the directory if it doesn't exist
    os.makedirs(os.path.dirname(file_path), exist_ok=True)

    # Write the row data to the text file
    with open(file_path, 'w', encoding='utf-8') as f:
      print(file_path)
      f.write(row_data)

def zip_and_download(typeoffile):
  # Zip the folders
  with zipfile.ZipFile(f'{typeoffile}.zip', 'w') as zip_obj:
    for folder_name, subfolders, filess in os.walk(f"/content/{typeoffile}/"):
        for file in filess:
            file_path = os.path.join(folder_name, file)
            zip_obj.write(file_path)
  # Download the zipped folders
  files.download(f'{typeoffile}.zip')





# def convert_large_csv_to_small_csv(inputcsvfile, outputcsvfile, percent):
#   large_df = pd.read_csv(inputcsvfile)
#   small_df = large_df.sample(frac=percent)  # 1% of the rows
#   small_df.to_csv(outputcsvfile, index=False)

In [ ]:
#To dowload the test data
download_test_data()


In [ ]:
df = pd.read_csv("appspecificdata.csv")




In [ ]:
df.shape




In [ ]:
df_notna = df[df["errorCode"].notna()]
df_notna.head()
df_notna.shape




In [ ]:
df_isna = df[df["errorCode"].isna()]
df_isna.head()
df_isna.shape


In [ ]:
sum = df_isna.shape[0]+df_notna.shape[0]
percentoferror = df_notna.shape[0] * 100 /sum
print(percentoferror)
percentofnoerror = df_isna.shape[0] * 100/sum
print(percentofnoerror)

In [ ]:
#df_isna.apply(lambda row, arg1: create_text_file(row, arg1), args=("noerror"), axis=1)

df_isna.apply(create_text_file, args=('noerror',), axis=1)

In [ ]:
df_notna.apply(create_text_file, args=('error',), axis=1)

In [ ]:

zip_and_download("error")
zip_and_download("noerror")